In [1]:
import pandas as pd
import sqlite3

# read data into DataFrame from db

con = sqlite3.connect("db/olympic_data.db")

sql = f"""
    SELECT * FROM athlete_events
    """

athlete_data = pd.read_sql(sql, con)
athlete_data.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,None
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,None
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,None
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,None


In [4]:
yearTeamGroup = athlete_data.groupby(["Year", "Team"])

groupAvg = yearTeamGroup.mean()
groupAvg

ID        Age      Height  \
Year Team                                                                  
1896 Australia                       35698.000000  22.000000         NaN   
     Australia/Great Britain         68525.000000  22.500000         NaN   
     Austria                         94272.375000  21.750000         NaN   
     Denmark                         74086.666667  25.933333         NaN   
     Ethnikos Gymnastikos Syllogos   64729.000000  10.000000         NaN   
     France                          59224.653846  19.789474         NaN   
     Germany                         75224.075269  24.313253  163.681818   
     Great Britain                   51443.869565  26.823529  188.000000   
     Great Britain/Germany           67321.000000  22.500000         NaN   
     Greece                          73489.135714  22.615385  175.666667   
     Greece-1                        56430.000000  20.000000         NaN   
     Greece-2                        95235.500000  17.000000         NaN   
     Greece-3                         1724.500000        NaN         NaN   
     Hungary                         90348.055556  22.642857         NaN   
     Italy                          101088.000000        NaN         NaN   
     Sweden                         111499.000000  21.000000         NaN   
     Switzerland                     76205.250000  30.000000         NaN   
     United States                   51846.259259  23.074074  179.875000   
1900 A North American Team           57763.750000  41.333333         NaN   
     Alcyon-6                        67718.000000        NaN         NaN   
     Alcyon-7                        67718.000000        NaN         NaN   
     Amateur Athletic Association    87707.200000  24.600000  176.500000   
     Amulet-3                        67211.000000  32.500000         NaN   
     Amulet-7                        67211.000000  32.500000         NaN   
     Argentina                       17641.000000  23.000000         NaN   
     Ariette-10                     102794.000000  62.000000         NaN   
     Ariette-8                      102794.000000  62.000000         NaN   
     Aschenbrodel                    99811.250000  44.000000         NaN   
     Australia                       88373.400000  21.800000  178.000000   
     Austria                         88953.708333  24.500000  176.500000   
...                                           ...        ...         ...   
2016 Switzerland-2                   81155.500000  29.000000  181.500000   
     Syria                           36088.571429  24.142857  185.333333   
     Tajikistan                      89658.571429  24.857143  179.142857   
     Tanzania                        78676.000000  24.428571  165.750000   
     Thailand                        76516.465517  25.965517  169.500000   
     Timor Leste                     68558.666667  26.000000  160.000000   
     Togo                            50406.600000  18.800000  158.000000   
     Tonga                           72158.142857  22.142857  177.857143   
     Trinidad and Tobago             52958.930233  26.418605  174.585366   
     Tunisia                         54037.353846  26.876923  180.761905   
     Turkey                          54976.428571  25.403361  175.184874   
     Turkmenistan                    60056.222222  22.444444  175.000000   
     Tuvalu                         120422.000000  24.000000  184.000000   
     Uganda                          57069.913043  23.130435  171.166667   
     Ukraine                         79853.160448  26.496269  174.902622   
     Ukraine-1                      111605.500000  24.500000  175.500000   
     Ukraine-2                       59552.500000  24.000000  174.000000   
     United Arab Emirates            44736.250000  27.833333  171.636364   
     United States                   63431.277539  26.217454  177.789398   
     United States Virgin Islands    80831.142857  27.285714  185.000000   
     United States-

In [5]:
groupAvg = groupAvg.reset_index()
groupAvg.head()

,Year,Team,ID,Age,Height,Weight
0,1896,Australia,35698.000000,22.000000,NaN,NaN
1,1896,Australia/Great Britain,68525.000000,22.500000,NaN,NaN
2,1896,Austria,94272.375000,21.750000,NaN,NaN
3,1896,Denmark,74086.666667,25.933333,NaN,NaN
4,1896,Ethnikos Gymnastikos Syllogos,64729.000000,10.000000,NaN,NaN


In [6]:
groupAvg = groupAvg[groupAvg["Team"] == "Australia"]

In [8]:
groupAvg = groupAvg.sort_values(by=["Year"])
groupAvg

,Year,Team,ID,Age,Height,Weight
0,1896,Australia,35698.000000,22.000000,NaN,NaN
28,1900,Australia,88373.400000,21.800000,178.000000,71.000000
210,1904,Australia,38199.666667,23.000000,NaN,NaN
288,1906,Australia,45276.900000,24.500000,172.200000,62.000000
518,1920,Australia,55245.551724,24.636364,175.733333,68.000000
592,1924,Australia,51992.655738,26.163636,177.318182,68.733333
682,1928,Australia,74067.620690,21.285714,176.375000,74.500000
766,1932,Australia,61888.850000,27.647059,175.285714,81.800000
842,1936,Australia,60679.166667,23.292683,174.000000,70.666667
980,1948,Australia,56830.871287,24.747368,179.833333,72.638889
